# Code to analyse 4 point scans vs photon count

Here this code is used to analyse the results of the dwell time vs number of repeats for the same photon count rate from the data. 

In [1]:
import numpy as np 
import pandas as pd 
import os 
import matplotlib.pyplot as plot

In [2]:
def load_data(file_path): 
    # load the ODMR data file in to an np array 
    data_array = np.loadtxt(file_path)
    return(data_array)

In [3]:
def load_params(file_path):
    # load in the params data. 
    params_data = pd.read_table(file_path, names = ['Parameter name', 'Parameter value']) 
    return(params_data)

In [4]:
def set_plot_params():
    # formats the plot parameters 
    plot.rcParams.update({'font.size' : 16})
    plot.rcParams.update({'figure.figsize' :[12, 8]})

In [5]:
def reshape_data(reference_data, odmr_data, params_values):
    '''Here we re-shape the reference and the ODMR datasets into a 3D array, 
    based on the number of line scans, frequnecy points and repeats per frequnecy 
    have been used to build the ODMR dataset.'''
    
    # Create variables with the frequnecy scan parameters. 
    num_of_line_scans = int(params_values[4])
    num_of_freqs =  int(1000*(params_values[1]-params_values[0])/params_values[2]) + 1
    points_per_freq = int(params_values[8])
    print(num_of_line_scans, num_of_freqs, points_per_freq)
    # Reshape the ODMR and reference datasets. 

    odmr_shape_3D = odmr_data[0:num_of_freqs*points_per_freq].reshape(num_of_line_scans, num_of_freqs, points_per_freq)
    reference_data_3D = reference_data[0:num_of_freqs*points_per_freq].reshape(num_of_line_scans, num_of_freqs, points_per_freq)
    
    return(odmr_shape_3D, reference_data_3D)

In [6]:
def normalise_data(reference_data, odmr_data):
    '''Here we normalise the ODMR dataset to the reference data.'''

    norm_odmr_data = odmr_data/reference_data
    return(norm_odmr_data)

In [7]:
def mean_data(odmr_data):
    # find the mean data 
    mean_odmr_data_points =  np.mean(odmr_data, axis = 2)
    mean_odmr_data_points_and_linescans =  np.mean(mean_odmr_data_points, axis = 0)
    return(mean_odmr_data_points, mean_odmr_data_points_and_linescans)

In [10]:
def run_analysis(date, scan, folder_path, four_point_data_set):
    '''I got bored of having to scroll up and down to input new numbers to analysis large datasets, 
    so I have split most of the code into this function to save myself.'''
    
    # Set the file names for all 4 data sets recorded from the experiment. 
    four_point_file_name_ODMR1 = date + four_point_data_set + '_ODMR1.dat'
    four_point_file_name_ODMR2 = date + four_point_data_set + '_ODMR2.dat'
    four_point_file_name_Ref1 = date + four_point_data_set  + '_Ref1.dat'
    four_point_file_name_Ref2 = date + four_point_data_set  + '_Ref2.dat'

    # Extract the ODMR and reference datasets from the data we are looking at. 
    four_point_odmr_data = load_data(folder_path + four_point_file_name_ODMR1)
    four_point_reference_data = load_data(folder_path + four_point_file_name_Ref1)

    # Get the parameters for the experiment. 
    four_point_params_file = load_params(folder_path + date  + four_point_data_set + '_Params.txt')
    four_point_params_values = four_point_params_file['Parameter value']

    # Need to have the four_freqs array layered as follows [F1, F2, F3, F4] due to riggering problems in the software.  
    four_freqs = [2.8592, 2.8612, 2.8763, 2.8783]

    # Reshape the data so that it can be used for the following script. 
    four_point_odmr_data, four_point_reference_data = reshape_data(four_point_reference_data, 
                                                               four_point_odmr_data, four_point_params_values)

    # Just a little bit of code to decide if we are finding the mean of linescans or point repeats. 
    store = [four_point_odmr_data.shape[0], four_point_odmr_data.shape[2]]
    if store[0] > store[1]:
        a = 0
    else: 
        a = 2

    # find the mean values for the four point ODMR data and reference data.
    # have to roll the data rund one position due to labview code + microwave egnerator interface. . 
    mean_odmr_data = np.mean(four_point_odmr_data, axis = a)
    mean_odmr_data = np.roll(mean_odmr_data, 1)

    total_counts = np.sum(four_point_odmr_data, axis = a)
    total_counts = np.roll(total_counts, 1)

    mean_ref_data = np.mean(four_point_reference_data, axis = a)
    mean_ref_data = np.roll(mean_ref_data, 1)

    # Normalise the full ODMR dataset and then find the mean of the normalised data for the four freqs. 
    norm_odmr_data = normalise_data(four_point_reference_data, four_point_odmr_data)
    mean_norm_odmr_data = np.mean(norm_odmr_data, axis = a)
    mean_norm_odmr_data = np.roll(mean_norm_odmr_data, 1)

    std_mean_norm_odmr = np.std(norm_odmr_data, axis = a)
    std_mean_norm_odmr = np.roll(std_mean_norm_odmr, 1)

    standard_error_norm_odmr = std_mean_norm_odmr/np.sqrt(four_point_odmr_data.shape[a])

    # Accuracy of Measurement 
    I_expected = np.array([0.96781868, 0.95413255, 0.951185, 0.96937859]) # The four normalised intensity points from a CW ODMR measurement. 
    delta_I = np.abs(I_expected - mean_norm_odmr_data)

    # Generate the error arrays for the mean of the ODMR and reference data. 
    four_points_ODMR_error = 2.2*np.sqrt(mean_odmr_data) # shot noise limit approximation
    four_points_reference_error = 2.2*np.sqrt(mean_ref_data) # shot noise limit approximation

    print(mean_norm_odmr_data[0])
    print(std_mean_norm_odmr[0])
    save_data(four_point_data_set, scan, date, four_point_params_values[8], 
          four_point_params_values[3]/1000, total_counts[0], 
          mean_odmr_data[0], four_points_ODMR_error[0], 
          mean_norm_odmr_data[0], std_mean_norm_odmr[0], 
          standard_error_norm_odmr[0], delta_I[0])

    return(1)

In [11]:
def save_data(file_name, scan_type, date, point_repeats, dwell_time, total_counts, 
             counts_per_point_repeat, shot_noise, norm_intensity, standard_deviation, 
             standard_error, accuracy):
    '''Save the data in a text file.'''
    
    save_folder = 'H:\\My Documents\\Strathclyde_Back_up\\Thesis and Planning\\Thesis_plots\\' + date + '\\'
    # Checking to see if the folder exists in which to save the data, and creating it if it does not
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        # Checking the output file exists and creating and initialising it if not
    
    file = save_folder + date + '_' +scan_type + '_statistics_measurement_for_I_'   
    
    for i in range(4):
        output_data_file = file + str(i+1) + '.dat'
        if os.path.isfile(output_data_file) == False:
            with open(output_data_file, "w") as wf:
                wf.write('File name'  + '\t' + 'Number of Repeats' + '\t' + 
                         'Dwell time (ms)' + '\t' + 'Total Counts recorded' + '\t' 
                         'Mean counts per single point (N)' + '\t' + '2.2 x sqrt(N)' + '\t' 
                         'Mean Normalised Counts (n)' + '\t' + 'Standard Deviation of n' + 
                         '\t' + 'Standard Error of n' + '\t' + 'Accuracy of measurement |I_CW - I_4point|' + '\n')
        
        print(standard_error[i])
        with open(output_data_file, "a") as wf:
            wf.write(date + '_' + file_name  + '\t' +
                 str(point_repeats) + "\t"  + str(dwell_time) + "\t" + 
                 str(total_counts[i]) + "\t"  +  
                 str(counts_per_point_repeat[i]) + "\t" + str(shot_noise[i]) + "\t" + 
                 str(norm_intensity[i]) + "\t"  + str(standard_deviation[i]) + "\t" + 
                 str(standard_error[i]) + '\t' +str(accuracy[i]) +'\n')       
        
    # Writing contrast dip description to the .dat file

### Code starts here 

In [83]:
# Load in the data file into an np array, and freqeuncy scan parameters 
# into a pandas array.

date = '20211217'
scan = 'Point_repeats' # Set if it was multiple point repeats or line scans. 
set_plot_params()

# Load in the data file into an np array, and freqeuncy scan parameters 
# into a pandas array.
folder_path = 'I:\\Science\\Physics-Nanobiophotonics\\Group\\User Data\\ODMR\\ODMR and imaging\\2021\\' + date + '\\'

four_point_data_set = '_4_point_referenced_odmr_70' #short hand for the file name. 

run_analysis(date, scan, folder_path, four_point_data_set)


1 4 1000
[0.97067355 0.95685372 0.95752591 0.97390552]
[0.02430636 0.02361739 0.02367757 0.0242454 ]
0.0007686347388002664
0.0007468472985511641
0.0007487504236993537
0.000766706878798168


1

In [25]:
print(four_point_params_values[8], 
          four_point_params_values[3]/1000, total_counts, 
          mean_odmr_data, four_points_ODMR_error, 
          mean_norm_odmr_data, standard_error_norm_odmr[0], 
          standard_error_norm_odmr[0])


NameError: name 'four_point_params_values' is not defined

In [26]:
print(mean_norm_odmr_data)

NameError: name 'mean_norm_odmr_data' is not defined